In [ ]:
!wget https://github.com/yohanesgultom/nlp-experiments/raw/master/data/ner/training_data.txt
!wget https://github.com/yohanesgultom/nlp-experiments/raw/master/data/ner/testing_data.txt

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import string
from multiprocessing import Pool

In [2]:
def make_iob(list_of_dicts):
    acceptable_characters = string.ascii_letters + string.digits + "-_*. "
    iob = list()
    for item in list_of_dicts:
        word = ''.join(c for c in item['word'] if c in acceptable_characters)
        iob.append("{} {}".format(word,item['tag']))
    return iob

In [3]:
def make_tagged_sentence(xml):
    xml = xml.split('\t')[0]
    tagged = list()
    objects = xml.replace('/ENAMEX>','').split('<')
    for obj in objects:
        #obj = obj.replace(':',' ').replace('.',' ').replace('(',' ').replace(')',' ')
        if obj.startswith('ENAMEX'):
            ner_type = obj.split('>')[0].replace('ENAMEX TYPE="','').replace('"','')
            words = obj.split('>')[1]
            words = words.split(' ')
            clean_word = words[0].translate(str.maketrans('', '', string.punctuation))
            if len(clean_word) > 0:
                tag = " B-{}".format(ner_type)
                tagged.append({'tag':tag,'word':clean_word})
            for i in range(len(words[1:])):
                clean_word = words[i+1].translate(str.maketrans('', '', string.punctuation))
                if len(clean_word) > 0:
                    tag = " I-{}".format(ner_type)
                    tagged.append({'tag':tag,'word':clean_word})
            #print(ner_type)
        else:
            obj = obj.split(' ')
            for word in obj:
                clean_word = word.translate(str.maketrans('', '', string.punctuation))
                if len(clean_word) > 0:
                    tag = 'O'
                    tagged.append({'tag':tag,'word':clean_word})
    return(tagged)

In [6]:
def make_iob_file(infilename):
    with open(infilename, 'r') as infile:
        sentences = infile.readlines()
        pool = Pool()
        tagged = pool.map(make_tagged_sentence,sentences)
        single_list = list()
        for tag in tagged:
            single_list.extend(tag)
        iob = make_iob(single_list)
    outfilename = infilename.replace('.txt','.iob')
    with open(outfilename,'w') as outfile:
        outfile.write("\n".join(str(item) for item in iob))

In [7]:
filenames = ['training_data.txt','testing_data.txt']

In [8]:
for filename in filenames:
    result = make_iob_file(filename)

In [9]:
iob

NameError: name 'iob' is not defined